In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, out, scene_ids, track_ids, agent_ids]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

### Visualize the batch of sequences

In [4]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 3

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
model = model.cuda()

# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        #for t in tqdm(range(100), desc='Interation'):

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        #print(y_pred.shape)
        #show_sample_batch(sample_batch, agent_id)

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


tensor(593.5699, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(515.7094, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(349.0583, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(436.7825, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(292.2936, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(295.0850, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(276.6018, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(335.3424, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(222.8814, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(171.2260, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(156.9842, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(123.6784, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(133.8347, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(115.2161, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(133.6079, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(119.3553, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(129.3462, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(76.8669

tensor(62.8253, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(86.6075, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.5695, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(73.7297, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(68.4032, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(77.2823, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(35.2452, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(27.6615, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.9415, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.2860, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(26.9312, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.1301, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.8136, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(30.7854, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.6402, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(30.8765, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.3093, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.7344, device='cuda:0'

tensor(45.2688, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(55.8269, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.3639, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(78.0970, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(75.7360, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(91.9053, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(47.2816, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(55.3805, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(49.4998, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(48.4651, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.4574, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.5026, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.3460, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.4646, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.7194, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.8194, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.6948, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.2555, device='cuda:0'

tensor(49.9531, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(47.4703, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(49.1317, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(45.0325, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(58.9976, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(49.1193, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(59.8025, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(56.1505, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.1934, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.4102, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(66.3517, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(68.3034, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(90.9621, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(52.6235, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(56.1520, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(35.3842, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(42.8494, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.2500, device='cuda:0'

tensor(32.9755, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(26.8503, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.3409, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.6045, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(24.2812, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(20.5350, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(27.4902, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(31.3366, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(41.2871, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(46.2141, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(49.4260, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(55.5687, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(78.8798, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(42.7795, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(38.5757, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(34.3594, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.2030, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.4143, device='cuda:0'

...

tensor(26.6394, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(40.7784, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(36.8619, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(48.5611, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(52.0792, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.4434, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(54.8204, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(45.8064, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.1291, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.3400, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.9161, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.5114, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.6326, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.1417, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.3494, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.8723, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.2186, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.0889, device='cuda:0', grad

tensor(46.7265, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(46.9381, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(51.8158, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(36.0158, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.9590, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.3068, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.0678, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.8277, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.7332, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.2027, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.4855, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.4756, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.7342, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.2349, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.7720, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.6337, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.8028, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.0564, device='cuda:0'

tensor(5.2188, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.4318, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.6576, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.0929, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.0145, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(6.1907, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.4282, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.3363, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.3876, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.1813, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.5441, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.5608, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.4554, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.9682, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.4919, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(6.0223, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.1755, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.1441, device='cuda:0', grad_fn=<SqrtBac

tensor(9.0489, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.0431, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.0453, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.5329, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.6267, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.0731, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.9494, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(17.6501, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.1356, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(26.6929, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.3903, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.4154, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.4234, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.7217, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.5327, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.9218, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.9188, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.2935, device='cuda:0', grad

tensor(3.1169, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.2119, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.6652, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.8086, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.4624, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.3353, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.0439, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.1235, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.0141, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.1368, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.8480, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.6915, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.7049, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.5844, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.0428, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.4009, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.2623, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.7355, device='cuda:0', grad_fn=<SqrtBac

In [5]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            #print(track_ids[63][15][0].item())
            #for i in range(30):
            #    print(track_ids[15][i][0].item())
            #print(agent_ids[15])
            #break
            
            #vehicle_index = 0
            #found = False
            #while not found:
            #    if track_ids[i][vehicle_index][j][0] == agent_id:
            #        found = True
            #    else:
            #        vehicle_index += 1
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                #for j in range(60):
                #    row.append(str(curr[j][29][0].item()) + ' ' + str(curr[j][29][1].item()))
                #data.append(row)
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                


         ID                  v1                  v2                  v3  \
0     10002  1771.5247802734375      315.2509765625    1709.60009765625   
1     10015   728.1885375976562      1220.580078125     723.72705078125   
2     10019   569.4191284179688  1238.9927978515625   564.6114501953125   
3     10028  1749.7376708984375   288.0892333984375  1676.6043701171875   
4      1003   2117.672607421875   645.9129638671875    2103.96630859375   
...     ...                 ...                 ...                 ...   
3195   9897  259.03472900390625   804.1195068359375  258.79730224609375   
3196     99   583.4259643554688  1140.2987060546875   579.0665893554688   
3197   9905  1782.6241455078125  423.80157470703125    1746.68212890625   
3198   9910   569.9649047851562  1280.8912353515625   566.1740112304688   
3199   9918   581.1257934570312   1144.459716796875   577.4783935546875   

                      v4                  v5                  v6  \
0      335.5980224609375     1